import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [3]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.8 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 1.6 MB/s eta 0:00:001.6 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317143 sha256=ab15dfab892d062c41d514b2e90d3aa0929d5f62203a5f813b2dfecb3a46e703
  Stored in directory: /home/user/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [24]:
import findspark
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, desc
from pyspark.sql.functions import when

findspark.init()

create local SparkSession

In [2]:
spark = SparkSession \
    .builder \
    .appName("PySpark_Basics") \
    .getOrCreate()

23/06/08 18:52:39 WARN Utils: Your hostname, user-ThinkBook-15-G2-ITL resolves to a loopback address: 127.0.1.1; using 192.168.0.152 instead (on interface wlp0s20f3)
23/06/08 18:52:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/08 18:52:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/08 18:52:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


read csv with inferschema

In [3]:
%%time
df4 = spark.read.options(inferSchema='True',delimiter=',') \
  .csv("ds_salaries.csv")

CPU times: user 1.62 ms, sys: 1.67 ms, total: 3.28 ms
Wall time: 3.99 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [4]:
%%time
df = spark.read.options(inferSchema='True',delimiter=',') \
  .csv("ds_salaries.csv")

CPU times: user 1.72 ms, sys: 1.76 ms, total: 3.48 ms
Wall time: 245 ms


write schema of scv on screen

In [5]:
%%time
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)

CPU times: user 1.24 ms, sys: 0 ns, total: 1.24 ms
Wall time: 1.46 ms


create schema of this scv

In [6]:
%%time
schema = StructType([ \
    StructField("", IntegerType(), nullable=True),        
    StructField("work_year", IntegerType(), nullable=True),
    StructField("experience_level", StringType(), nullable=True),
    StructField("employment_type", StringType(), nullable=True),
    StructField("job_title", StringType(), nullable=True),
    StructField("salary", IntegerType(), nullable=True),
    StructField("salary_currency", StringType(), nullable=True),
    StructField("salary_in_usd", IntegerType(), nullable=True),
    StructField("employee_residence", StringType(), nullable=True),
    StructField("remote_ratio", IntegerType(), nullable=True),
    StructField("company_location", StringType(), nullable=True),
    StructField("company_size", StringType(), nullable=True)
  ])

CPU times: user 19 µs, sys: 18 µs, total: 37 µs
Wall time: 40.1 µs


restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [7]:
%%time
df = spark.read.csv("ds_salaries.csv", schema=schema, header=True)
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|   |work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [8]:
%%time
df.printSchema()

root
 |-- : integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)

CPU times: user 722 µs, sys: 686 µs, total: 1.41 ms
Wall time: 2.07 ms


now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [9]:
%%time
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|   |work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [10]:
%%time
df.toPandas()

CPU times: user 89.5 ms, sys: 38.7 ms, total: 128 ms
Wall time: 341 ms


,,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [13]:
%%time
df_job_titles = df.select("job_title").distinct()

CPU times: user 2.23 ms, sys: 1.66 ms, total: 3.88 ms
Wall time: 36.7 ms


print all rows from df_job_titles without truncating jobs

In [14]:
%%time
df_job_titles.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [18]:
%%time
df_analytic = df.groupBy("job_title") \
        .agg(max("salary_in_usd").alias("max_salary"), \
         avg("salary_in_usd").alias("avg_salary"), \
         min("salary_in_usd").alias("min_salary") \
     ) 

CPU times: user 3.73 ms, sys: 0 ns, total: 3.73 ms
Wall time: 32.1 ms


print all rows from df_analytic without trancating jobs

In [19]:
%%time
df_analytic.show(truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |276000    |139724.5          |56000     |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |170000    |92203.0           |19609     |
|Data Engineering Manager                |174000    |123227.2          |59303     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [31]:
%%time
windowSpec  = Window.orderBy(desc("avg_salary"))

df_analytic = df_analytic.withColumn("row_number",row_number().over(windowSpec))

CPU times: user 1.81 ms, sys: 0 ns, total: 1.81 ms
Wall time: 8.23 ms


print all data from df_analytic

In [32]:
%%time
df_analytic.show(truncate=False)

+----------------------------------+----------+------------------+----------+----------+
|job_title                         |max_salary|avg_salary        |min_salary|row_number|
+----------------------------------+----------+------------------+----------+----------+
|Data Analytics Lead               |405000    |405000.0          |405000    |1         |
|Principal Data Engineer           |600000    |328333.3333333333 |185000    |2         |
|Financial Data Analyst            |450000    |275000.0          |100000    |3         |
|Principal Data Scientist          |416000    |215242.42857142858|148261    |4         |
|Director of Data Science          |325000    |195074.0          |130026    |5         |
|Data Architect                    |266400    |177873.9090909091 |90700     |6         |
|Applied Data Scientist            |380000    |175655.0          |54238     |7         |
|Analytics Engineer                |205300    |175000.0          |135000    |8         |
|Data Specialist     

23/06/08 19:25:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:25:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:25:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:25:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:25:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:25:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [33]:
%%time
df_analytic = df_analytic.select("row_number", "job_title", "max_salary", "avg_salary", "min_salary")

CPU times: user 4.69 ms, sys: 178 µs, total: 4.87 ms
Wall time: 16 ms


print df_analytic now

In [34]:
%%time
df_analytic.show(truncate=False)

+----------+----------------------------------+----------+------------------+----------+
|row_number|job_title                         |max_salary|avg_salary        |min_salary|
+----------+----------------------------------+----------+------------------+----------+
|1         |Data Analytics Lead               |405000    |405000.0          |405000    |
|2         |Principal Data Engineer           |600000    |328333.3333333333 |185000    |
|3         |Financial Data Analyst            |450000    |275000.0          |100000    |
|4         |Principal Data Scientist          |416000    |215242.42857142858|148261    |
|5         |Director of Data Science          |325000    |195074.0          |130026    |
|6         |Data Architect                    |266400    |177873.9090909091 |90700     |
|7         |Applied Data Scientist            |380000    |175655.0          |54238     |
|8         |Analytics Engineer                |205300    |175000.0          |135000    |
|9         |Data Spec

23/06/08 19:27:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:27:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:27:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:27:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:27:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/06/08 19:27:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [36]:
%%time
df_exp_lvl = df.groupBy("experience_level") \
        .agg(max("salary_in_usd").alias("biggest_salary")) 

CPU times: user 3.18 ms, sys: 0 ns, total: 3.18 ms
Wall time: 11.1 ms


print here df_exp_lvl

In [37]:
%%time
df_exp_lvl.show()

+----------------+--------------+
|experience_level|biggest_salary|
+----------------+--------------+
|              EX|        600000|
|              MI|        450000|
|              EN|        250000|
|              SE|        412000|
+----------------+--------------+

CPU times: user 1.7 ms, sys: 0 ns, total: 1.7 ms
Wall time: 136 ms


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [58]:
%%time
df_best = df.alias("df").join(df_exp_lvl, \
                     (df.salary_in_usd == df_exp_lvl.biggest_salary) & \
                     (df.experience_level == df_exp_lvl.experience_level), "inner")

df_best = df_best.select("", "df.experience_level", "biggest_salary", "employee_residence")

CPU times: user 5.23 ms, sys: 0 ns, total: 5.23 ms
Wall time: 19.9 ms


print df_best

In [59]:
%%time
df_best.show()

+---+----------------+--------------+------------------+
|   |experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 33|              MI|        450000|                US|
| 37|              EN|        250000|                US|
| 63|              SE|        412000|                US|
| 97|              MI|        450000|                US|
|252|              EX|        600000|                US|
+---+----------------+--------------+------------------+

CPU times: user 2.64 ms, sys: 0 ns, total: 2.64 ms
Wall time: 138 ms


drop duplicates if exist by experience_level

In [66]:
%%time
df_best = df_best.dropDuplicates(["experience_level"])

CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 2.76 ms


print df_best

In [67]:
%%time
df_best.show()

+---+----------------+--------------+------------------+
|   |experience_level|biggest_salary|employee_residence|
+---+----------------+--------------+------------------+
| 37|              EN|        250000|                US|
|252|              EX|        600000|                US|
| 33|              MI|        450000|                US|
| 63|              SE|        412000|                US|
+---+----------------+--------------+------------------+

CPU times: user 1.99 ms, sys: 0 ns, total: 1.99 ms
Wall time: 223 ms


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [84]:
%%time
df_new_best = df_best.select(col("*"), when(df_best.experience_level  == "MI","middle")
                  .when(df_best.experience_level == "SE","senior")
                  .otherwise("Null").alias("level_category"))
df_new_best = df_new_best.select("level_category", "biggest_salary", "employee_residence")

CPU times: user 4.17 ms, sys: 0 ns, total: 4.17 ms
Wall time: 15.6 ms


print df_new_best

In [85]:
%%time
df_new_best.show()

+--------------+--------------+------------------+
|level_category|biggest_salary|employee_residence|
+--------------+--------------+------------------+
|          Null|        250000|                US|
|          Null|        600000|                US|
|        middle|        450000|                US|
|        senior|        412000|                US|
+--------------+--------------+------------------+

CPU times: user 2.08 ms, sys: 0 ns, total: 2.08 ms
Wall time: 158 ms


write df_new_best like 1.csv and load then it to df_final

In [91]:
%%time
df_new_best.write.csv("1.csv")

df_final = spark.read.csv("1.csv")

CPU times: user 4.29 ms, sys: 0 ns, total: 4.29 ms
Wall time: 316 ms


print df_final

In [120]:
%%time
df_final.show()

+------+------+---+
|   _c0|   _c1|_c2|
+------+------+---+
|  Null|250000| US|
|  Null|600000| US|
|middle|450000| US|
|senior|412000| US|
+------+------+---+

CPU times: user 2.61 ms, sys: 0 ns, total: 2.61 ms
Wall time: 95.3 ms


filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [121]:
%%time
df_final = df_final.filter("_c0 != \'Null\'")
df_final = df_final.join(df, \
                   (df_final._c1 == df.salary_in_usd) & \
                   (df_final._c2 == df.employee_residence), "inner")

columns_to_exclude = ["_c0", "_c1", "_c2"]
df_final = df_final.select([col for col in df.columns if col not in columns_to_exclude])

CPU times: user 2.92 ms, sys: 4.07 ms, total: 6.99 ms
Wall time: 28.9 ms


print df_final

In [122]:
%%time
df_final.show()

+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|   |work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
| 33|     2020|              MI|             FT|  Research Scientist|450000|            USD|       450000|                US|           0|              US|           M|
| 63|     2020|              SE|             FT|      Data Scientist|412000|            USD|       412000|                US|         100|              US|           L|
| 97|     2021|              MI|             FT|Financial Data An...|450000|            USD|       450000|                US|         100|              US|

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [125]:
%%time
biggest_salary_in_usd = df_final.select(max("salary_in_usd")).first()[0]
print(biggest_salary_in_usd)

450000
CPU times: user 2.58 ms, sys: 0 ns, total: 2.58 ms
Wall time: 106 ms


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system